<a href="https://colab.research.google.com/github/cosminluciantuiasi/dissertation/blob/feature%2Ffacial-recognition-part1/face_recognition_part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#/content/drive/MyDrive/Colab Notebooks/disertation/dataset